In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


In [3]:
#importing packages

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import image_dataset_from_directory
import warnings

warnings.filterwarnings('ignore')

2025-06-10 15:31:20.202191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749569480.391755      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749569480.448653      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
#checking for duplications

import os
def get_all_image_paths(folder_path):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.jpg')):
                image_paths.append(os.path.join(root, file))
    return set(image_paths)

train_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Train')
val_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Validation')
test_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Test')

# Intersections
print("Train ∩ Val:", len(train_paths & val_paths))
print("Train ∩ Test:", len(train_paths & test_paths))
print("Val ∩ Test:", len(val_paths & test_paths))

Train ∩ Val: 0
Train ∩ Test: 0
Val ∩ Test: 0


In [6]:
#loading image data
def load_1stimage_data(path):
    img_size = (256, 256)
    batch_size = 64

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Train",
        label_mode="binary",  
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Validation",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True  
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Test",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False  
    )


    return train_ds,val_ds ,test_ds

train,val,test=load_1stimage_data('/kaggle/input/deepfake-and-real-images')


Found 140002 files belonging to 2 classes.


I0000 00:00:1749572943.979407      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [7]:
print("Class names and their labels:", train.class_names)

Class names and their labels: ['Fake', 'Real']


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

model = Sequential([
    Input((256, 256, 3)),
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1),
    RandomContrast(0.1),

    Conv2D(64, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(64, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    GlobalAveragePooling2D(),

    Dense(512, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])


In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)              │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip_1 (RandomFlip)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_1 (RandomRotation)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_1 (RandomZoom)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation_1                 │ (None, 256, 256, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast_1 (RandomContrast)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 256, 256, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 256, 256, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 10,338,881 (39.44 MB)

 Trainable params: 10,331,201 (39.41 MB)

 Non-trainable params: 7,680 (30.00 KB)

In [14]:
from tensorflow.keras.optimizers import Adam

custom_adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint(filepath='best_model.keras',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

earlystop=EarlyStopping(monitor='val_accuracy',patience=5,verbose=1,restore_best_weights=True)

lr_scheduler=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


model.fit(train,epochs=10,validation_data=val,callbacks=[checkpoint, earlystop, lr_scheduler])


Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 0.7538 - loss: 0.5324
Epoch 1: val_accuracy improved from -inf to 0.74475, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 677s 303ms/step - accuracy: 0.7538 - loss: 0.5323 - val_accuracy: 0.7447 - val_loss: 0.8141 - learning_rate: 0.0010
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 0.9366 - loss: 0.1668
Epoch 2: val_accuracy improved from 0.74475 to 0.87382, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 663s 303ms/step - accuracy: 0.9366 - loss: 0.1668 - val_accuracy: 0.8738 - val_loss: 0.3413 - learning_rate: 0.0010
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 0.9527 - loss: 0.1249
Epoch 3: val_accuracy improved from 0.87382 to 0.89269, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 662s 303ms/step - accuracy: 0.9527 - loss: 0.1249 - val_accuracy: 0.8927 - val_loss: 0.2429 - learning_rate: 0.0010
Epoch 4/10
2188/2188 ━━━━

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

y_true = []
y_pred = []

for images, labels in test:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype("int32").flatten())  

y_true = np.array(y_true)
y_pred = np.array(y_pred)

test_loss, test_accuracy = model.evaluate(test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━

In [ ]:
!zip -r folder.zip /kaggle/working/best_model.keras

  adding: kaggle/working/best_model.keras (deflated 45%)


In [ ]:
from IPython.display import FileLink

FileLink(r'folder.zip')

/kaggle/working/folder.zip